In [2]:
# open and read input file
with open("kendrick_lamar_lyrics.txt", "r", encoding="utf-8") as f:
    lyrics = f.read()

In [3]:
# check how many characters there are in the dataset
len(lyrics)

554065

In [4]:
# take a look at the first 1000 characters
print(lyrics[:1000])

sit still and close your eyes smoke to it
what's behind the other door oh-ohh
no more silence no more silence
don't kill this thing we got called love don't shoot
just searching for the perfect shot

i used to write rhymes all day and all night
when y'all was playing playstation, my pencil was erasing lines
my conscience only knew what's half-tight
at 3:14, it's time to get me a slice my nigga
this is a dog's fight my nigga
the soundtrack to life my nigga
kendrick lamar, his momma called him that
he watched house party and ate apple jacks
he sold sega games, his cousin sold crack
he pumped reeboks, his uncles pumped packs
punk fake, jump-shot, ball hit the back
ball dreams of being point guard was off limits jack
that's because these compton streets was built not to win
you killed the nigga, i stole a bible, is that a sin
part of me though, i'm searching for answers just searching for the perfect shot
the good kid from the ugly city that's mad that he's had some
where is the love

when

In [5]:
# check all the unique characters that occur in the dataset
chars = sorted(list(set(lyrics)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !"&'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzàâãéíïúе– 
86


## Simple tokeniser for the text/lyrics


In [6]:
# create a mapping from characters to integers (encoder) and vice-versa (decoder)
str_to_int = {char: i for i, char in enumerate(chars)}
int_to_str = {i: char for i, char in enumerate(chars)}

In [7]:
# encoder takes a string which outputs a list of integers ,
# characters in string are converted to int via lookup table
encode = lambda s: [str_to_int[c] for c in s]

# decoder takes a list of integers and outputs a string, integers converted via lookup table
decode = lambda l: "".join([int_to_str[i] for i in l])

In [8]:
print(encode("helloo there"))
print(decode(encode("helloo there")))

[57, 54, 61, 61, 64, 64, 1, 69, 57, 54, 67, 54]
helloo there


In [9]:
# encode the entire dataset from string to integers
import torch

data = torch.tensor(encode(lyrics), dtype=torch.long)
print(data.shape, data.type)
# the first 1000 characters from our dataset converted using our encoder mapping (from str to int)
print(data[:1000])

torch.Size([554065]) <built-in method type of Tensor object at 0x000001366FC0D370>
tensor([68, 58, 69,  1, 68, 69, 58, 61, 61,  1, 50, 63, 53,  1, 52, 61, 64, 68,
        54,  1, 74, 64, 70, 67,  1, 54, 74, 54, 68,  1, 68, 62, 64, 60, 54,  1,
        69, 64,  1, 58, 69,  0, 72, 57, 50, 69,  5, 68,  1, 51, 54, 57, 58, 63,
        53,  1, 69, 57, 54,  1, 64, 69, 57, 54, 67,  1, 53, 64, 64, 67,  1, 64,
        57,  9, 64, 57, 57,  0, 63, 64,  1, 62, 64, 67, 54,  1, 68, 58, 61, 54,
        63, 52, 54,  1, 63, 64,  1, 62, 64, 67, 54,  1, 68, 58, 61, 54, 63, 52,
        54,  0, 53, 64, 63,  5, 69,  1, 60, 58, 61, 61,  1, 69, 57, 58, 68,  1,
        69, 57, 58, 63, 56,  1, 72, 54,  1, 56, 64, 69,  1, 52, 50, 61, 61, 54,
        53,  1, 61, 64, 71, 54,  1, 53, 64, 63,  5, 69,  1, 68, 57, 64, 64, 69,
         0, 59, 70, 68, 69,  1, 68, 54, 50, 67, 52, 57, 58, 63, 56,  1, 55, 64,
        67,  1, 69, 57, 54,  1, 65, 54, 67, 55, 54, 52, 69,  1, 68, 57, 64, 69,
         0,  0, 58,  1, 70, 68, 54, 5

## Create our train and validation datasets


In [10]:
# split 90% of our data into train and have remaining 10% as our validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
len(train_data)

498658

In [12]:
len(val_data)

55407

In [13]:
# use block_size / context length interchangeably
block_size = 8
train_data[: block_size + 1]

tensor([68, 58, 69,  1, 68, 69, 58, 61, 61])

given a tensor above, there are 8 training examples within a sample of size blocksize. <br>
E.g : <br>
x = 68 , y = 58 <br>
x = [68,58], y = 69 <br>
x = [68,58,69] , y = 1 <br>
etc until the 8th training example within this sample <br>
x = [68,58,69,1,68,69,58,61], y = 61 <br>


In [14]:
x = train_data[:block_size]
# y is technically our target for the input of x, thus it is ofset by 1
y = train_data[1 : block_size + 1]

In [15]:
x

tensor([68, 58, 69,  1, 68, 69, 58, 61])

In [16]:
y

tensor([58, 69,  1, 68, 69, 58, 61, 61])

In [17]:
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when in put is {context} the target is: {target}")

when in put is tensor([68]) the target is: 58
when in put is tensor([68, 58]) the target is: 69
when in put is tensor([68, 58, 69]) the target is: 1
when in put is tensor([68, 58, 69,  1]) the target is: 68
when in put is tensor([68, 58, 69,  1, 68]) the target is: 69
when in put is tensor([68, 58, 69,  1, 68, 69]) the target is: 58
when in put is tensor([68, 58, 69,  1, 68, 69, 58]) the target is: 61
when in put is tensor([68, 58, 69,  1, 68, 69, 58, 61]) the target is: 61


In [18]:
torch.manual_seed(42)
batch_size = 4  # how many independent sequences will we process in parallel
block_size = 8  # what is the maximum context length for predictions

In [19]:
def get_batch(split):
    # generate a small batch of data for inputs x and targets y
    data = train_data if split == "train" else val_data

    # generate a tensor of random integers, that represent the starting position of each sequence of data
    batch_start_indicies = torch.randint(high=(len(data) - block_size), size=(batch_size,))

    # creates a tensor: x and y where each element is a sequence of block_size, stack the 1-D tensors as rows
    # creating a batch_size x block_size tensors (e.g 4x8 tensor)
    x = torch.stack([data[index : index + block_size] for index in batch_start_indicies])
    y = torch.stack([data[index + 1 : index + block_size + 1] for index in batch_start_indicies])
    return x, y

In [20]:
xb, yb = get_batch('train')
print(f"Batch size = {batch_size} \nBlock size = {block_size} \n")

print('inputs: ')
print(xb.shape)
print(xb, '\n')

print('targets: ')
print(yb.shape)
print(yb, '\n')


Batch size = 4 
Block size = 8 

inputs: 
torch.Size([4, 8])
tensor([[58, 52, 57,  1, 63, 58, 56, 56],
        [64, 72,  1, 59, 70, 68, 69,  1],
        [55, 70, 52, 60, 58, 63,  5,  1],
        [32,  5, 62,  1, 51, 61, 54, 68]]) 

targets: 
torch.Size([4, 8])
tensor([[52, 57,  1, 63, 58, 56, 56, 50],
        [72,  1, 59, 70, 68, 69,  1, 72],
        [70, 52, 60, 58, 63,  5,  1, 62],
        [ 5, 62,  1, 51, 61, 54, 68, 68]]) 



In [21]:
for b in range(batch_size): # iterate through the batch dimension (batches)
    for t in range(block_size): # iterate through the time dimension (each character/element/number in the batch)
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is {target}")
    print("\n")

When input is [58] the target is 52
When input is [58, 52] the target is 57
When input is [58, 52, 57] the target is 1
When input is [58, 52, 57, 1] the target is 63
When input is [58, 52, 57, 1, 63] the target is 58
When input is [58, 52, 57, 1, 63, 58] the target is 56
When input is [58, 52, 57, 1, 63, 58, 56] the target is 56
When input is [58, 52, 57, 1, 63, 58, 56, 56] the target is 50


When input is [64] the target is 72
When input is [64, 72] the target is 1
When input is [64, 72, 1] the target is 59
When input is [64, 72, 1, 59] the target is 70
When input is [64, 72, 1, 59, 70] the target is 68
When input is [64, 72, 1, 59, 70, 68] the target is 69
When input is [64, 72, 1, 59, 70, 68, 69] the target is 1
When input is [64, 72, 1, 59, 70, 68, 69, 1] the target is 72


When input is [55] the target is 70
When input is [55, 70] the target is 52
When input is [55, 70, 52] the target is 60
When input is [55, 70, 52, 60] the target is 58
When input is [55, 70, 52, 60, 58] the targ

Our 4x8 tensor contains 32 training examples, which are completely independent

In [22]:
# our input to the transformer 
xb

tensor([[58, 52, 57,  1, 63, 58, 56, 56],
        [64, 72,  1, 59, 70, 68, 69,  1],
        [55, 70, 52, 60, 58, 63,  5,  1],
        [32,  5, 62,  1, 51, 61, 54, 68]])

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        # maps each token in the sequence to its next-token prediction in the form of logits
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(
            idx
        )  # (B,T,C) , where C = channel = vocab size

        if targets is None:
            # if no target, just get the logits
            loss = None
        else:
            # since we have a multi-dimensional input, PyTorch cross_entropy function expects the logit tensor shape to be (B*T,C) and not (B,T,C)
            # and the target tensor to be 1D. Therefore we have to reshape our logit and target tensors
            B, T, C = logits.shape
            logits = logits.view(B * T, C) # want it to be 2-D Tensor
            targets = targets.view(B*T)  # want it to be 1-D Tensor

            # measures the quality of the logits w.r.t the targets,
            # how well are we predicting the next character based on the logits
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) tensor of indicies in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # only consider last element in the time dimension, becomes (B,C)
            logits = logits[:,-1,:] 
            # apply softmax to get proababilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution, only gets 1 prediction so becomes (B,1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [24]:
model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb,yb)


In [25]:
logits.shape

torch.Size([32, 86])

In [26]:
loss

tensor(4.9753, grad_fn=<NllLossBackward0>)

we are expecting a loss of -ln(1/vocab_size)

In [27]:
import math
from math import log
print(f"We expect the loss to be about {-log(1/vocab_size)}")

We expect the loss to be about 4.454347296253507


In [28]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


í–v L'szïU.е7/bR
Z1g6'sl)HwPp(–"âXo//Dâ?JIwNGqígàl(ZéG-YY4a6A9е,Oú–CàKJ3")LNkm1é9ASpL/qKJk/D(WdfB234


generated output is random because our model hasnt been trained

## Model Training

In [29]:
# intialise optimiser
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [30]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model(xb,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()

print(loss.item())

2.471187114715576


In [31]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


te yof is theaYSAl(Msin ache fomancenet'mery gand l n boucanarr g, m

iedole
i g m g, bul e
id sithe joren ccr sthandovorlorall
iman ly ait t, me mp, m ite intul me, bl tcourinck
but my, tot jus bas a tht h m ton'm t icillundy gil daroma juc gif, as
atinith a t bewery dediali wheambugumy zis tien'sthatheretherkel wige itanndpeledorigen wi-Rt hemine RUpodbr c mey hino aut peg ithed a lor, gachortin ilache, piler o
An'aixts buloon, thur o hearomyolala yontwathtoverbeen dne t anke?44
titr bean'myou


## Matrix multiplication in self-attention

In [10]:
# example 1: inefficient method using loops 
torch.manual_seed(42)
# Batch, time, channels
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [11]:
# bow = bag of words
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C) tensor
        xbow[b,t] = torch.mean(xprev, dim = 0) # averaging out on the t-dimension , to get 1D C-vector

In [12]:
# 0th batch element
x[0]

tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047],
        [-0.7521,  1.6487],
        [-0.3925, -1.4036],
        [-0.7279, -0.5594],
        [-0.7688,  0.7624]])

In [20]:
xbow[0]
# we can see that as we iterate over the t-dimensino each value is an average with the previous values e.g the at t = 8, the 2 values in the vector
# are the average down the column.

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])

using loops to calculate our running averages across the t-dimension is inefficient , especially as our tensors get bigger. We can use matrix multiplciation to speed up this calculation

In [17]:
# example 2: using matrix multiplcation
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("a=")
print(a, "\n")
print("b=")
print(b, "\n")
print("c=")
print(c, "\n")

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) 

b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 

c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]]) 



In [22]:
# use torch.tril to createa a lower triangular matrix of size nxn, we will be using this to make sure the future tokens do not interact with the past tokens as we are trying to predict the future tokens
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [29]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
# normalise the rows and make each row sum to 1. Lets us calculate the average of each row.
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("a=")
print(a, "\n")
print("b=")
print(b, "\n")
print("c=")
print(c, "\n")

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 

b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 

c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]]) 



In [30]:
# create our xbow using matrix multiplication
weights = torch.tril(torch.ones(T,T))
weights = weights / weights.sum(dim=1, keepdim=True)
# our "weights-tensor" would be equivalent to our "a-tensor" in the example above, our "b-tensor" would be "x-tensor"
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [32]:
xbow2 = weights @ x # weights = (T,T) @ x = (B,T,C) --> PyTorch will create a batch-dimension in weights tensor to be (B,T,T) and then after batched matrix-mult of (B,T,T) @ (B,T,C) ==> (B,T,C)
xbow2.shape

torch.Size([4, 8, 2])

In [34]:
xbow2[0]

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])

In [35]:
torch.allclose(xbow,xbow2)

True

In [36]:
xbow[0], xbow2[0]

(tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]))

we can see that xbow2 is identical to xbow but we used a more efficient method of using batched matrix-multiplcation instead of loops to compute the tensors.

In [46]:
# example 3 :  with softmax
tril = torch.tril(torch.ones((T,T)))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [69]:
weights = torch.zeros((T,T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [70]:
weights = F.softmax(weights, dim=-1)
# softmax exponentiates values, e^0 = 1, and e^-inf = 0, useful for when our weights are data dependent and not statically made
xbow3 = weights @ x
xbow3[0]

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])

In [71]:
torch.allclose(xbow,xbow3)

True

summary of this section : we can do weighted aggregations of our past elements by using matrix multiplciation of a lower triangular matrix, elements in the lower triangle are how much the words relate to each other